# 自動微分

自動微分についてちゃんとまとめる。

In [1]:
import torch

## `requires_grad`

pytorchでは、`torch.Tensor`(以下tensorと呼ぶ)の演算に対して計算グラフを構築することで、自動微分をサポートしている。

tensorは`requires_grad`という`bool`型の属性を持っており、これが`True`の場合、そのtensorを用いた演算が計算グラフに追加される。`False`の場合は追加されない。微分が必要ないと分かっている場合は`False`にしてメモリを節約した方が良い。VRAMは貴重。  
`requires_grad`はデフォルトで`False`になっている。

In [2]:
x1 = torch.tensor([0.1, 0.2, 0.3])
x1.requires_grad

False

`True`にしたかったら初期化時に引数に渡す。

In [3]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x1.requires_grad

True

後から変更することもできる。

In [4]:
x1.requires_grad = False
x1.requires_grad

False

こんなメソッドもある。

In [5]:
x1.requires_grad_(True) # デフォルト: True
x1.requires_grad

True

`float`じゃないとダメ

In [23]:
try:
    x1 = torch.tensor([1, 2, 3], requires_grad=True)
except Exception as e:
    print(e)

Only Tensors of floating point and complex dtype can require gradients


演算によって作られた新たなtensorの`requires_grad`は、演算に関わったtensorの`requires_grad`に依存する。演算に関わったtensorの中に`requires_grad`が`True`のものが1つでもあれば、新たなtensorの`requires_grad`は`True`になる。

In [6]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
x2 = torch.tensor([0.4, 0.5, 0.6], requires_grad=False)

z1 = x1 * 2
print(z1.requires_grad) # True

z2 = x2 * 2
print(z2.requires_grad) # False

z3 = x1 * x2
print(z3.requires_grad) # True

True
False
True


## `backward()`

微分を行うメソッド。

`backward()`を呼び出すと、そのtensorから計算グラフを遡り（逆伝播）、結果を各tensorの`grad`属性に格納する。  
別の言い方をすると、そのtensorに関わった全てのtensorで変微分を行い、結果を各tensorの`grad`属性に格納する。

In [15]:
x1 = torch.tensor(2., requires_grad=True)
x2 = torch.tensor(5., requires_grad=False)
z = x1 * x2
z.backward()
print(x1.grad)
print(x2.grad)

tensor(5.)
None


$$
x_1 = 2, x_2 = 5 \\
z = x_1x_2 \\
\frac{\partial z}{\partial x_1} = x_2 = 5 \\
$$

なので、`x1.grad`は5になる。`x2`は`requires_grad`が`False`なのでそもそも勾配を計算しない。

tensorはスカラーでなくても良い。

In [17]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
z.backward()
print(x1.grad)

tensor([2., 2., 2.])


ただし`backward()`はスカラーに対してしか使えない。

In [22]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = x1 * 2
try:
    z.backward()
except Exception as e:
    print(e)

grad can be implicitly created only for scalar outputs


2回`backward()`を呼び出すと怒られる。

In [35]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
z.backward() # 1回目
try:
    z.backward() # 2回目
except Exception as e:
    print(e)

Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.


`retain_graph=True`を指定すると、その次の`backward()`が通る。  
複数回逆伝播を行うと`grad`は加算される。

In [39]:
x1 = torch.tensor([0.1, 0.2, 0.3], requires_grad=True)
z = (x1 * 2).sum()
for _ in range(10):
    z.backward(retain_graph=True)
print(x1.grad)

tensor([20., 20., 20.])
